[모델 ANN]
1. 2개의 은닉층을 사용하는 ANN 구조
- 절단 정규 분포 (tf.truncated_normal)로 초기화, ReLU 활성화 함수 사용
2. 손실함수 : cross-entropy
3. 옵티마이저 : Gradient Descent Optimizer (learning rate=0.3)
4. 학습 epoch=30
- batch size=32

In [312]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [313]:
# Data
def load_csv(file_path):

    # CSV 파일 로드
    data = np.loadtxt(file_path, delimiter=',', skiprows=1)
    
    # 입력 변수와 목표 변수 분리 ('fetal_health'를 목표 변수로 설정)
    x_data = data[:, :-1]  # 마지막 열 제외 (입력 변수)
    y_data = data[:, -1].astype(int)  # 마지막 열 (목표 변수)
    
    # 목표 변수를 one-hot 인코딩으로 변환
    with tf.Session() as sess:
        y_data = sess.run(tf.one_hot(y_data - 1, depth=3))

    return x_data, y_data

def split_data(x_data, y_data):
    # 첫 200개의 데이터를 테스트 데이터로 사용
    x_test = x_data[:200]
    y_test = y_data[:200]
    
    # 나머지 데이터를 학습 및 검증 데이터로 사용
    x_rest = x_data[200:]
    y_rest = y_data[200:]
    
    # 학습 데이터와 검증 데이터로 분할 예시 (전체의 약 12.5%를 검증 데이터로 사용)
    x_train, x_val, y_train, y_val = train_test_split(x_rest, y_rest, test_size=0.125, random_state=42)

    # 표준화 (스케일링)
    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_val = sc.transform(x_val)
    x_test = sc.transform(x_test)
    
    return x_train, x_val, x_test, y_train, y_val, y_test

In [314]:
x_data, y_data = load_csv('fetal_health.csv')
x_train, x_val, x_test, y_train, y_val, y_test = split_data(x_data, y_data)

In [315]:
learning_rate = 0.3
num_epochs = 30      # 학습횟수
batch_size = 32    # 배치개수
display_step = 10    # 손실함수 출력 주기
input_size = x_train.shape[1]    # 21
hidden1_size = 128  # 첫 번째 은닉층의 크기
hidden2_size = 64  # 두 번째 은닉층의 크기
output_size = y_train.shape[1]   # 3

In [316]:
x = tf.placeholder(tf.float32, shape=[None, input_size])
y = tf.placeholder(tf.float32, shape=[None, output_size])

In [317]:
def build_ANN(x):
    # 첫 번째 은닉층
    W1 = tf.Variable(tf.truncated_normal([input_size, hidden1_size], stddev=0.1))
    b1 = tf.Variable(tf.constant(0.1, shape=[hidden1_size]))
    H1_output = tf.nn.relu(tf.matmul(x, W1) + b1)

    # 두 번째 은닉층
    W2 = tf.Variable(tf.truncated_normal([hidden1_size, hidden2_size], stddev=0.1))
    b2 = tf.Variable(tf.constant(0.1, shape=[hidden2_size]))
    H2_output = tf.nn.relu(tf.matmul(H1_output, W2) + b2)

    # 출력층
    W_output = tf.Variable(tf.truncated_normal([hidden2_size, output_size], stddev=0.1))
    b_output = tf.Variable(tf.constant(0.1, shape=[output_size]))
    logits = tf.matmul(H2_output, W_output) + b_output

    return logits

predicted_value = build_ANN(x)

In [318]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predicted_value, labels=y))
train_step = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

In [319]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # 학습 루프
    total_batch = len(x_train) // batch_size
    for epoch in range(num_epochs):
        average_loss = 0
        for i in range(total_batch):
            # 미니배치 데이터 준비
            start = i * batch_size
            end = start + batch_size
            batch_x, batch_y = x_train[start:end], y_train[start:end]
            
            # 학습 수행 및 손실 계산
            _, current_loss = sess.run([train_step, loss], feed_dict={x: batch_x, y: batch_y})
            average_loss += current_loss / total_batch
        
        # 매 epoch마다 손실 및 검증 정확도 출력
        if epoch % display_step == 0:
            # 학습 데이터 손실
            print("반복(Epoch): %d, 손실 함수(Loss): %f" % ((epoch+1), average_loss))
            
            # 검증 데이터에 대한 정확도 계산
            correct_prediction_val = tf.equal(tf.argmax(predicted_value, 1), tf.argmax(y, 1))
            accuracy_val = tf.reduce_mean(tf.cast(correct_prediction_val, "float"))
            print("검증 정확도(Validation Accuracy): %f" % accuracy_val.eval(feed_dict={x: x_val, y: y_val}))
    
    # 테스트 데이터에 대한 최종 정확도 계산
    correct_prediction_test = tf.equal(tf.argmax(predicted_value, 1), tf.argmax(y, 1))
    accuracy_test = tf.reduce_mean(tf.cast(correct_prediction_test, "float"))
    print("최종 테스트 정확도(Test Accuracy): %f" % (accuracy_test.eval(feed_dict={x: x_test, y: y_test})))


반복(Epoch): 1, 손실 함수(Loss): 0.386120
검증 정확도(Validation Accuracy): 0.900415
반복(Epoch): 11, 손실 함수(Loss): 0.143442
검증 정확도(Validation Accuracy): 0.908714
반복(Epoch): 21, 손실 함수(Loss): 0.092350
검증 정확도(Validation Accuracy): 0.917012
최종 테스트 정확도(Test Accuracy): 0.880000
